In [1]:
from datasets import load_dataset

ravis_dataset = load_dataset("ravistech/clinical-trial-llm-Open_condition_Cleaned_dup_NCT_ID")

print(ravis_dataset)
print(ravis_dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['metadata', 'data', 'criteria', 'NCT_ID', '__index_level_0__'],
        num_rows: 213877
    })
    validation: Dataset({
        features: ['metadata', 'data', 'criteria', 'NCT_ID', '__index_level_0__'],
        num_rows: 26735
    })
    test: Dataset({
        features: ['metadata', 'data', 'criteria', 'NCT_ID', '__index_level_0__'],
        num_rows: 26735
    })
})
{'metadata': '{\n"NCT_ID" : "NCT04411810",\n"Brief_Title" : "SpotCheck: Comparison of Enhanced Telemedicine Versus In-person Evaluation for the Diagnosis of Skin Cancer",\n\n"Official_title" : "SpotCheck: Comparison of Enhanced Telemedicine Versus In-person Evaluation for the Diagnosis of Skin Cancer",\n\n"Conditions" : ["Skin Cancer"],\n\n"Interventions" : ["Device: Nevisense 3.0", "Procedure: Skin biopsy", "Device: Dermlite Cam", "Device: Barco Demetra"],\n\n"Location_Countries" : ["United States"],\n\n"Study_Design" : {\n"Study_Type" : "INTERVENTIONAL",\n"Phase" :

In [2]:
# import re
# def fix_invalid_json(input_str):
#     ## add double quotes around elements inside square brackets if not already quoted
#     fixed_str = re.sub(r'(?<=\[)([^\[\],]+)(?=\])', lambda x: '"' + x.group(0).strip() + '"', input_str)
    
#     ## add double quotes around words in Conditions and Interventions
#     fixed_str = re.sub(r'(?<=\[)([^\"\]]+?)(?=\])', lambda x: '"' + x.group(0).strip().replace(", ", '", "') + '"', fixed_str)
    
#     ## fix key-value pairs inside Interventions
#     fixed_str = re.sub(r'"([A-Za-z]+): ([A-Za-z0-9\s]+)"', r'"\1: \2"', fixed_str)
    
#     # fix dictionary keys
#     fixed_str = re.sub(r'(?<!")(\b[A-Za-z_]+\b)(?=\s*:)', r'"\1"', fixed_str)
    
#     return fixed_str

In [3]:
# !pip install json_repair
# import chromadb
# from datasets import load_dataset
# import json
# import json_repair
# from sentence_transformers import SentenceTransformer
# from unidecode import unidecode

# client = chromadb.PersistentClient(path="./clinical_trials_chroma")
# model = SentenceTransformer("malteos/scincl")
# collection = client.get_or_create_collection("clinical_trials_studies")

# ravis_dataset = load_dataset("ravistech/clinical-trial-llm-cancer-restructure")

# def embed_studies_from_dataset(dataset, batch_size=32):
#     batch_texts = []       
#     batch_metadata = []    
#     batch_documents = []   
#     batch_ids = []         
#     index = 1
#     length = len(dataset['train'])
    
#     for study in dataset['train']:
#             metadata = json_repair.loads(study['metadata'])
#             title = metadata.get('Official_title', '') or metadata.get('Brief_Title', '')
#             detailed_description = study.get('data', '')

#             # Skip if no valid title or detailedDescription
#             if not title or not detailed_description:
#                 continue
            
#             concatenated_text = unidecode(f"{title} [SEP] {detailed_description}")
#             batch_texts.append(concatenated_text)
#             batch_metadata.append({
#                 "nctId": metadata.get("NCT_ID", "unknown"),
#                 "officialTitle": title,
#                 "detailedDescription": detailed_description,
#                 "jsonMetadata": json.dumps(metadata, ensure_ascii=True)
#             })
#             batch_documents.append(json.dumps({
#                 "metadata": metadata,
#                 "description": study.get('data', ''),
#                 "criteria": study.get('criteria', '')
#                 },ensure_ascii=True))
#             batch_ids.append(metadata.get("NCT_ID", "unknown"))

#             # When batch size is reached, process the batch
#             if len(batch_texts) == batch_size:
#                 process_batch(batch_texts, batch_documents, batch_ids, batch_metadata)
#                 print(f"Processed {len(batch_texts)} studies. {index}/{length}")
#                 # Clear the batches
#                 batch_texts.clear()
#                 batch_documents.clear()
#                 batch_metadata.clear()
#                 batch_ids.clear()
#             index += 1

#     if batch_texts:
#         process_batch(batch_texts, batch_documents, batch_ids, batch_metadata)

# def process_batch(texts, documents, ids, metadatas):
#     embeddings = model.encode(texts, batch_size=len(texts))
#     collection.add(
#         embeddings=embeddings,
#         documents=documents,
#         metadatas=metadatas,
#         ids=ids
#     )
#     print(f"Processed and added batch of {len(texts)} studies.")

# # adjust batch_size to fit in your gpu memory
# embed_studies_from_dataset(ravis_dataset, batch_size=750)
# print("Embedding and storing complete!")

In [4]:
!pip install json_repair
import chromadb
from datasets import load_dataset
import json
import json_repair
from sentence_transformers import SentenceTransformer
from unidecode import unidecode

client = chromadb.PersistentClient(path="./clinical_trials_chroma_all")
model = SentenceTransformer("malteos/scincl")
collection_studies = client.get_or_create_collection("clinical_trials_studies")
collection_criteria = client.get_or_create_collection("clinical_trials_criteria")
ravis_dataset = load_dataset("ravistech/clinical-trial-llm-Open_condition_Cleaned_dup_NCT_ID")

def embed_studies_from_dataset(dataset, batch_size=32):
    batch_texts = []       
    batch_criteria = []    # New list for criteria texts
    batch_metadata = []    
    batch_documents = []   
    batch_ids = []         
    index = 1
    length = len(dataset['train'])
    
    for study in dataset['train']:
            metadata = json_repair.loads(study['metadata'])
            try:
                title = metadata.get('Official_title', '') or metadata.get('Brief_Title', '')
            except:
                print(metadata)
                continue
            detailed_description = study.get('data', '')
            criteria_text = study.get('criteria', '')  # Get criteria text

            # Skip if no valid title, description or criteria
            if not title or not detailed_description or not criteria_text:
                continue
            
            # Prepare texts for both collections
            concatenated_text = unidecode(f"{title} [SEP] {detailed_description}")
            criteria_text = unidecode(criteria_text)
            
            batch_texts.append(concatenated_text)
            batch_criteria.append(criteria_text)
            batch_metadata.append({
                "nctId": metadata.get("NCT_ID", "unknown"),
                "officialTitle": title,
                "detailedDescription": detailed_description,
                "jsonMetadata": json.dumps(metadata, ensure_ascii=True)
            })
            batch_documents.append(json.dumps({
                "metadata": metadata,
                "description": study.get('data', ''),
                "criteria": study.get('criteria', '')
                },ensure_ascii=True))
            batch_ids.append(metadata.get("NCT_ID", "unknown"))

            if len(batch_texts) == batch_size:
                process_batch(batch_texts, batch_criteria, batch_documents, batch_ids, batch_metadata)
                print(f"Processed {len(batch_texts)} studies. {index}/{length}")
                batch_texts.clear()
                batch_criteria.clear()
                batch_documents.clear()
                batch_metadata.clear()
                batch_ids.clear()
            index += 1

    if batch_texts:
        process_batch(batch_texts, batch_criteria, batch_documents, batch_ids, batch_metadata)

def process_batch(texts, criteria_texts, documents, ids, metadatas):
    # Process and add to studies collection
    study_embeddings = model.encode(texts, batch_size=len(texts))
    collection_studies.add(
        embeddings=study_embeddings,
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    
    # Process and add to criteria collection
    criteria_embeddings = model.encode(criteria_texts, batch_size=len(criteria_texts))
    collection_criteria.add(
        embeddings=criteria_embeddings,
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    print(f"Processed and added batch of {len(texts)} studies to both collections.")
embed_studies_from_dataset(ravis_dataset, batch_size=750)
print("Embedding and storing complete!")

Processed and added batch of 750 studies to both collections.
Processed 750 studies. 750/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 1500/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 2250/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 3000/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 3750/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 4500/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 5250/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 6000/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 6750/213877
Processed and added batch of 750 studies to both collections.
Processed 750 studies. 7500/213877
Processed and added batch of 75